<a href="https://colab.research.google.com/github/rubyclaguna/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/Copy_of_LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
# Upload data
import pandas as pd 
from google.colab import files
uploaded = files.upload()

Saving house-votes-84.data to house-votes-84.data


In [0]:
import numpy as np 
from scipy import stats

In [0]:
# no specific column names assigned, soi set header=None
df = pd.read_csv('house-votes-84.data', header = None)
print(df.shape)
df.head()

(435, 17)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
# Assign column names 
df = df.rename ({0: 'party',           
                 1: 'handicapped_infants',  
                 2: 'water_project_cost_sharing',
                 3: 'adoption_of_the_budget_resolution', 
                 4: 'physician_fee_freeze', 
                 5: 'el_salvador_aid', 
                 6: 'religious_groups_in_schools', 
                 7: 'anti_satellite_test_ban', 
                 8: 'aid_to_nicaraguan_contras', 
                 9: 'mx_missile',
                10: 'immigration',
                11: 'synfuels_corporation_cutback',
                12: 'education_spending',
                13: 'superfund_right_to_sue',
                14: 'crime',
                15: 'duty_free_exports',
                16: 'export_administration_act_south_africa'}, axis = 1)

In [0]:
df.head()

,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 17 columns):
party                                     435 non-null object
handicapped_infants                       435 non-null object
water_project_cost_sharing                435 non-null object
adoption_of_the_budget_resolution         435 non-null object
physician_fee_freeze                      435 non-null object
el_salvador_aid                           435 non-null object
religious_groups_in_schools               435 non-null object
anti_satellite_test_ban                   435 non-null object
aid_to_nicaraguan_contras                 435 non-null object
mx_missile                                435 non-null object
immigration                               435 non-null object
synfuels_corporation_cutback              435 non-null object
education_spending                        435 non-null object
superfund_right_to_sue                    435 non-null object
crime                      

In [0]:
df.isnull().sum()

party                                     0
handicapped_infants                       0
water_project_cost_sharing                0
adoption_of_the_budget_resolution         0
physician_fee_freeze                      0
el_salvador_aid                           0
religious_groups_in_schools               0
anti_satellite_test_ban                   0
aid_to_nicaraguan_contras                 0
mx_missile                                0
immigration                               0
synfuels_corporation_cutback              0
education_spending                        0
superfund_right_to_sue                    0
crime                                     0
duty_free_exports                         0
export_administration_act_south_africa    0
dtype: int64

In [0]:
import numpy as np
# Change '?' to NaN to determine missing values. 
df = df.replace ('?', np.NaN)

In [0]:
df.isnull().sum()

party                                       0
handicapped_infants                        12
water_project_cost_sharing                 48
adoption_of_the_budget_resolution          11
physician_fee_freeze                       11
el_salvador_aid                            15
religious_groups_in_schools                11
anti_satellite_test_ban                    14
aid_to_nicaraguan_contras                  15
mx_missile                                 22
immigration                                 7
synfuels_corporation_cutback               21
education_spending                         31
superfund_right_to_sue                     25
crime                                      17
duty_free_exports                          28
export_administration_act_south_africa    104
dtype: int64

In [0]:
# Drop missing values. Should be enough to data to work with if these values are dropped. 
df = df.dropna()

In [0]:
df= df.replace({'n':0, 'y':1})
df.head()

,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
5,democrat,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
8,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
19,democrat,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1
23,democrat,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,1
25,democrat,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1


In [0]:
# Seperate by party in order to compare data
df_dem = df[df.party == "democrat"]
print(df_dem.shape)
df_dem.head()

(124, 17)


,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
5,democrat,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
19,democrat,1,1,1,0,0,0,1,1,1,0,1,0,0,0,1,1
23,democrat,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,1
25,democrat,1,0,1,0,0,0,1,1,1,1,0,0,0,0,1,1
26,democrat,1,0,1,0,0,0,1,1,1,0,1,0,0,0,1,1


In [0]:
df_rep = df[df.party == "republican"]
print(df_rep.shape)
df_rep.head()

(108, 17)


,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
8,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
28,republican,1,0,0,1,1,0,1,1,1,0,0,1,1,1,0,1
30,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,0
33,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
35,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,0


In [0]:
group = df_all.groupby(["party"]).mean()
group = group.T
group

party,democrat,republican
handicapped_infants,0.588710,0.212963
water_project_cost_sharing,0.451613,0.472222
adoption_of_the_budget_resolution,0.854839,0.157407
physician_fee_freeze,0.048387,0.990741
el_salvador_aid,0.201613,0.953704
religious_groups_in_schools,0.443548,0.870370
anti_satellite_test_ban,0.766129,0.268519
aid_to_nicaraguan_contras,0.830645,0.148148
mx_missile,0.790323,0.138889
immigration,0.532258,0.574074


In [0]:
# Add 3rd column to compare difference 
group['diff'] = group['democrat'] - group['republican']

In [0]:
group

party,democrat,republican,diff
handicapped_infants,0.588710,0.212963,0.375747
water_project_cost_sharing,0.451613,0.472222,-0.020609
adoption_of_the_budget_resolution,0.854839,0.157407,0.697431
physician_fee_freeze,0.048387,0.990741,-0.942354
el_salvador_aid,0.201613,0.953704,-0.752091
religious_groups_in_schools,0.443548,0.870370,-0.426822
anti_satellite_test_ban,0.766129,0.268519,0.497611
aid_to_nicaraguan_contras,0.830645,0.148148,0.682497
mx_missile,0.790323,0.138889,0.651434
immigration,0.532258,0.574074,-0.041816


In [0]:
import seaborn as sns
from matplotlib import style
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

In [0]:
# Null Hypothesis that Dem support is 0; no support. 
stats.ttest_1samp(df_dem['handicapped_infants'], 0)

Ttest_1sampResult(statistic=13.268715971389689, pvalue=1.7333710512218448e-25)

In [0]:
# Null Hypothesis that Dem neither support not oppose the issue
stats.ttest_1samp(df_dem['handicapped_infants'], .5)

Ttest_1sampResult(statistic=1.9993955573326936, pvalue=0.04776946299893786)